# MNIST with Tensorboard

Last update: 2022-07-21, Tuan Do

Example of ML experiment with MNIST with tensorboard. We will assume that tensorboard is already running in the background or will be run after this notebook. See other instructions for how to create a local instance of tensorboard or upload to tensorboard dev.

How to save checkpoints: https://keras.io/api/callbacks/model_checkpoint/


In [4]:
import tensorflow as tf
import datetime
import tensorboard
import os
print(tensorboard.__version__)
print(tensorboard.__path__)

2.5.0
['/home/tdo/miniconda3/envs/jupyterhub3/lib/python3.8/site-packages/tensorboard']


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])




In [11]:
model_name = 'mnist_cnn_v1'
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# check points
checkpoint_filepath = os.path.join("/models/",model_name)+"/"+model_name

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_freq='epoch',
    save_best_only=True)

# location where we will save our logs. This should also be the location that tensorboard watches. 
# In this case to not overwrite the logs, we will name our experiment after the date and time it was started
log_dir = os.path.join("/logs/",model_name)
epochs = 10

# use this to log into tensorboard and look at performance in real time
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)



In [12]:
model.fit(x=x_train, 
          y=y_train, 
          epochs=epochs, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback,model_checkpoint_callback])

# save the model after fitting
model.save(checkpoint_filepath)

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2189 - accuracy: 0.9359 - val_loss: 0.0993 - val_accuracy: 0.9695
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0976 - accuracy: 0.9699 - val_loss: 0.0879 - val_accuracy: 0.9727
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0694 - accuracy: 0.9780 - val_loss: 0.0967 - val_accuracy: 0.9696
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0531 - accuracy: 0.9832 - val_loss: 0.0707 - val_accuracy: 0.9789
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0447 - accuracy: 0.9856 - val_loss: 0.0645 - val_accuracy: 0.9804
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0375 - accuracy: 0.9877 - val_loss: 0.0688 - val_accuracy: 0.9814
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0310 - accuracy: 0.9892 - val_loss: 0.0854 - val_accuracy:

In [10]:
# list what is in the log files
!ls $log_dir

train  validation
